In [1]:
from src.guess.guess import ConfigType, Guess, Config
from src.domain.datasets.UnixCommandDataset import UnixCommandDataset  # Register dataset
from src.domain.datasets.ProjectEulerDataset import ProjectEulerDataset
from src.domain.datasets.BringUpDataset import BringUpDataset
from src.domain.datasets.HumanEvalDataset import HumanEvalDataset

prediction = None

config = Config(ConfigType.QWEN_X862ARM64.get_path())
guess = Guess(config=config)

# results = guess.guess()

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-12 19:38:50,288 - src.domain.models.QwenModel - INFO - Initializing QwenModel with ahmedheakl/ex19_qwen2.5-1.5b-1m-stack-16kcw on device mps


Loading Qwen model: ahmedheakl/ex19_qwen2.5-1.5b-1m-stack-16kcw


2025-05-12 19:38:52,966 - QwenModel - INFO - Initialized QwenModel on device: mps:0
2025-05-12 19:38:52,967 - src.domain.models.QwenModel - INFO - Model initialization completed in 2.68 seconds


Source JSONL: data/processed/X86/BringUp_x86.jsonl
Target JSONL: data/processed/ARM64/BringUp_arm.jsonl
Loading file: data/processed/X86/BringUp_x86.jsonl
Loaded 31 entries from data/processed/X86/BringUp_x86.jsonl
Loading file: data/processed/ARM64/BringUp_arm.jsonl
Loaded 31 entries from data/processed/ARM64/BringUp_arm.jsonl


In [2]:
from src.helpers.torch_unpicker import CPU_Unpickler

with open("predl.pkl", "rb") as f:
    result = CPU_Unpickler(f).load()
    


In [3]:
print(result.keys())

dict_keys(['fy-shuffle.s', 'bloom-filter.s', 'strange.s', 'topo-sort.s', 'weekday.s', 'knights-tour.s', 'murmur-hash.s', 'spirograph.s', 'gcd-list.s', 'skeleton.s', 'simple-grep.s', 'pi-calc.s', 'indirect-test.s', 'flood-fill.s', 'priority-queue.s', 'bubble-sort.s', 'ackermann.s', 'checkers.s', 'spelt2num.s', 'totient.s', 'cipher.s', 'mersenne.s', 'heapsort.s', 'sieve.s', 'hanoi.s', 'natlog.s', 'rle-compress.s', 'quine.s', 'mandelbrot.s'])


In [4]:
from src.sketch.sketch import Sketch

sketch = Sketch(config, guess.model)

In [5]:
from src.sketch.fixes import fix_duplicate_sections, fix_missing_sections
from src.helpers.model import PredictionResult
import torch

In [6]:
for id, pred in result.items():
    pred_len = len(pred.pred)
    fixed_pred = PredictionResult(
        instance_id=id,
        source=torch.tensor([pred.source], dtype=torch.int64),
        pred=torch.tensor([pred.pred], dtype=torch.int64),
        confidence=pred.confidence[:pred_len],
        alignments=pred.alignments[:pred_len],
    )

    fixed_pred = fix_duplicate_sections(sketch, fixed_pred)
    fixed_pred = fix_missing_sections(sketch, fixed_pred)

3705
3709
7418
4019
4019
8038
978
978
1956
4743
4743
9486
3098
3098
6196
4814
4814
9628
3677
3677
7354
2964
2964
5928
2060
2098
4196
526
529
1058
4140
4140
8280
2330
2330
4660
1899
1899
3798
3722
3732
7464
3681
3681
7362
3329
3329
6658
3369
3373
6746
3085
3085
6170
Missing section: LBB0_7
4087
4095
8190
Missing section: LBB0_222
4970
4970
9940


2025-05-12 19:38:55,616 - src.domain.models.QwenModel - INFO - Starting inference with temperature=0.7, max_length=512
2025-05-12 19:38:55,616 - src.domain.models.QwenModel - INFO - Input shape: torch.Size([1, 102])
`torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to eager attention. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


AttributeError: 'list' object has no attribute 'device'